In [1]:
import os
import warnings

warnings.simplefilter(action="ignore")
os.environ["GRPC_VERBOSITY"] = "NONE"

# Prerequisites

Please make sure your environmental variables and dependencies are ready to use LLM services.

In [2]:
import json

from dotenv import load_dotenv

load_dotenv("../../.env_api")

True

# Import modules

In langrila, the module to chat with LLM and the module to call tools are completely separated. FunctionalChat class is the combination of those two.

- `XXXChatModule`: Only focuses on doing conversation with LLM
- `XXXFunctionCallingModule`: Only focuses on calling tools
- `XXXFunctionalChat`: The combination of the two. FunctionCallingModule works at first and then ChatModule performs. If any tool is not provided, this module behaves as just ChatModule.

In [3]:
from langrila.gemini import GeminiFunctionalChat
from langrila.openai import OpenAIFunctionalChat

# Structured output

OpenAI Chat Completion and Gemini support json mode generation.

### For OpenAI Chat Completion

In [4]:
chat_openai = OpenAIFunctionalChat(api_key_env_name="API_KEY")

In [5]:
prompt_template = """# INSTRUCTION
Extract fruit names from the following text and output with the following JSON format. Don't generate JSON code block.

# TEXT
{prompt}

# OUTPUT JSON FORMAT
{{
    "fruits": ["xxx", "xxx", "xxx"]
}}
"""

In [6]:
prompt = "This store sells apples, oranges, and bananas."
prompt = prompt_template.format(prompt=prompt)
response = chat_openai.run(prompt=prompt, model_name="gpt-4o-2024-08-06", json_mode=True)
json.loads(response.message.content[0].text)

{'fruits': ['apples', 'oranges', 'bananas']}

Pydantic schema is also available with 2 arguments: `json_mode` and `response_schema`.

In [7]:
from pydantic import BaseModel


class Step(BaseModel):
    explanation: str
    output: str


class MathReasoning(BaseModel):
    steps: list[Step]
    final_answer: str

In [8]:
chat_openai = OpenAIFunctionalChat(api_key_env_name="API_KEY")

In [9]:
prompt = "how can I solve 8x + 7 = -23"

response = chat_openai.run(
    prompt=prompt,
    model_name="gpt-4o-2024-08-06",
    json_mode=True,  # also needed
    response_schema=MathReasoning,
    system_instruction="You are a helpful math tutor. Guide the user through the solution step by step.",
)

# resposne message is in JSON string that can be converted to specified schema
resposne_json = json.loads(response.message.content[0].text)
MathReasoning(**resposne_json)

MathReasoning(steps=[Step(explanation='Start by isolating the term with the variable, 8x, on one side of the equation. To do this, subtract 7 from both sides of the equation.', output='8x + 7 - 7 = -23 - 7'), Step(explanation='Simplify both sides of the equation. On the left side, 7 - 7 cancels out, leaving 8x. On the right side, -23 - 7 equals -30.', output='8x = -30'), Step(explanation='To solve for x, divide both sides of the equation by 8 to get x by itself.', output='(8x)/8 = (-30)/8'), Step(explanation='Simplify the right-hand side. -30 divided by 8 simplifies to -15/4 or -3.75.', output='x = -3.75')], final_answer='x = -3.75')

### For Gemini

In [10]:
from typing_extensions import TypedDict


class ResponseSchema(TypedDict):
    fruits: list[str]

In [11]:
gemini = GeminiFunctionalChat(api_key_env_name="GEMINI_API_KEY")

In [12]:
prompt = "This store sells apples, oranges, and bananas."

response = gemini.run(
    prompt=prompt,
    model_name="gemini-1.5-flash",
    json_mode=True,
    response_schema=ResponseSchema,
)

[PydanticDeprecatedSince20]:The `schema` method is deprecated; use `model_json_schema` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.9/migration/


In [13]:
json.loads(response.message.content[0].text)

{'fruits': ['apples', 'oranges', 'bananas']}